In [ ]:
### Imports
import os
import sys
import numpy as np
import pandas as pd
import click
import logging
import pickle
import cv2
import resource
import random
from datetime import datetime
from pathlib import Path
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

# import supported models
from tensorflow.keras.applications import ResNet50, MobileNet, MobileNetV2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical

# import preprocessing functions for the models
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.applications.mobilenet import preprocess_input as preprocess_input_mobile_net
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_input_mobile_net_v2

from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.compat.v1 import set_random_seed
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression


def get_project_path():
    try:
        # running from a python file"
        project_path = Path(os.path.dirname(os.path.abspath(__file__)), os.pardir)
        is_file = True
        is_colab = False
    except NameError as e:
        # not running from a file (ie console or notebook)
        project_path = Path(os.getcwd(), os.pardir)
        is_file = False
        try:
            # running on google colab
            from google.colab import drive
            # connect to google drive
            drive.mount('/content/gdrive')
            project_path = Path("/content/gdrive/My Drive/Colab Notebooks/setdetection/")
            is_colab = True
        except ModuleNotFoundError:
            # not running on google colab
            is_colab = False
            output_path = project_path / "output"
    return project_path, is_file, is_colab


# get project path, and flag if script runs in a file
project_path, is_file, is_colab = get_project_path()
output_path = project_path / "output"
results_csv_path = output_path / 'results.csv'

# columns for results csv. This list is used to create the results dataframe
result_columns = ['run_id', 'layer_id', 'layer_name', 'model', 'nr_images', 'normalize', 'train_score', 'val_score']

# add project root to pythonpath
sys.path.insert(0, str(project_path / "src"))

# import custom packages
from utils.identify import *    
from utils.log import *
from utils.click_utils import conditionally_decorate
from set_cardgame.dataset import *


def get_or_create_results_file(results_csv_path, columns, sep=";"):
    try:
        df_results = pd.read_csv(results_csv_path, sep=sep)
    except FileNotFoundError:
        # create new results dataframe dictionary to store results
        df_results = pd.DataFrame(columns=columns)
    return df_results


def fit_linear_model(linear_model, X_train, y_train, X_val, y_val):
    '''
        Args:
            model_type (str): {'sklearn_logistic_regression', 'keras_dense'}
    '''
    
    if linear_model == "sklearn_logistic_regression":
        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)        
        train_acc = clf.score(X_train, y_train)
        val_acc = clf.score(X_val, y_val)
    elif linear_model == "keras_dense":
        logger.info(f"linear_model: {linear_model}")
        
        # convert to one hot
        Y_train_onehot = to_categorical(y_train)
        Y_val_onehot = to_categorical(y_val)

        logger.info(f"X_train.shape: {X_train.shape}")
        logger.info(f"y_train.shape: {y_train.shape}")
        logger.info(f"Y_train_onehot.shape: {Y_train_onehot.shape}")
        
        logger.info(f"X_val.shape: {X_val.shape}")
        logger.info(f"y_val.shape: {y_val.shape}")
        logger.info(f"Y_val_onehot.shape: {Y_val_onehot.shape}")

        batch_size=128
        epochs = 20
        input_dim = X_train.shape[1]
        output_dim = 81 # nr classes
        
        logger.info(f"batch_size: {batch_size}")
        logger.info(f"epochs: {epochs}")        
        logger.info(f"input_dim: {input_dim}")
        logger.info(f"output_dim: {output_dim}")

        model = Sequential() 
        model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 

        model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
        history = model.fit(X_train, Y_train_onehot, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, Y_val_onehot)) 
        train_acc = history.history['acc'][-1]
        logger.info(f"val_acc, final epoch: {history.history['val_acc'][-1]}")        
        score = model.evaluate(X_val, Y_val_onehot, verbose=0)
        val_acc = score[1]
    else:
        logger.error("unsupported linear_model")
    return train_acc, val_acc


# decorate with click commands if the script runs from a file
@conditionally_decorate(click.command(), is_file)
@conditionally_decorate(click.option('--normalize', default="none", help='how to normalize data, possible values: {"standard", "minimax"}.'),  is_file)
@conditionally_decorate(click.option('--nr_images', default=81, help='Number of images to generate for training'), is_file)
@conditionally_decorate(click.option('--model', default="resnet", help='which model to use for embeddings: {"resnet", "mobilenet", "mobilenet_v2"}.'),  is_file)
@conditionally_decorate(click.option('--linear_model', default="sklearn_logistic_regression", help='which linear model to use: {"sklearn_logistic_regression", "keras_dense"}.'),  is_file)
def train(normalize="none", nr_images=81, model="resnet", linear_model="sklearn_logistic_regression"):
    # create id for this run
    run_id = f'{model}_{normalize}_{nr_images}_{datetime.now().strftime("%d%m%Y-%H%M%S")}'
    
    # setup logging
    logger = setup_logger(level=logging.INFO, logfile=output_path / f"log_{run_id}.txt")
    
    # make results repeatable
    seed = 42
    random.seed(seed)  # `python` built-in pseudo-random generator
    np.random.seed(seed)  # numpy pseudo-random generator
    set_random_seed(seed)  # tensorflow pseudo-random generator
    logger.info(f"np.random.random(): {np.random.random()}")

    # create id for this run
    run_id = datetime.now().strftime("%d%m%Y-%H%M%S")

    logger.info("======")
    logger.info(f"starting run {run_id}")
    logger.info(f"parameter: normalize: {normalize}")
    logger.info(f"parameter: nr_images: {nr_images}")
    logger.info(f"parameter: model: {model}")
    logger.info(f"project_path: {project_path}")
    logger.info(f"output_path: {output_path}")
    logger.info(f"is_file: {is_file}")
    logger.info(f"is_colab: {is_colab}")

    # initialize intermediate model
    if model == "resnet":
        model_instance = ResNet50(include_top=False, weights='imagenet', pooling=None, input_shape=(96, 128, 3))
        preprocess_input_function = preprocess_input_resnet50
    elif model == "mobilenet":
        model_instance = MobileNet(include_top=False, weights='imagenet', pooling=None, input_shape=(96, 128, 3))
        preprocess_input_function = preprocess_input_mobile_net
    elif model == "mobilenet_v2":
        model_instance = MobileNetV2(include_top=False, weights='imagenet', pooling=None, input_shape=(96, 128, 3))
        preprocess_input_function = preprocess_input_mobile_net_v2
    else:
        logger.error(f"unsupported model: {model}")
        return

    # load training and validation data
    X_train, y_train, X_val, y_val = load_dataset(nr_images=nr_images, output_path=None)

    # apply preprocessing for the specified model
    X_train = preprocess_input_function(X_train)
    X_val = preprocess_input_function(X_val)

    # nr samples
    m_train = X_train.shape[0]
    m_val = X_val.shape[0]
    
    logger.debug("X_train.shape:", X_train.shape)
    logger.debug("X_val.shape:", X_val.shape)
    logger.debug("m_train:", m_train)
    logger.debug("m_val:", m_val)

    # get results dataframe, or if it is not there create it
    df_results = get_or_create_results_file(results_csv_path, result_columns)

    # keep track of memory usage
    mem_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

    # loop randomly through layers
    nr_layers = len(model_instance.layers)
    random_index = np.random.choice(range(1, nr_layers), size=nr_layers-1, replace=False)
    for i, layer_id in enumerate(random_index):
        logger.info("===")
        logger.info(f"process layer: {i} - {layer_id}")
        # log memory usage for each iteration
        mem_usage1 = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        logger.info(f"Memory usage: {mem_usage1}, diff: {mem_usage1-mem_usage}")
        mem_usage = mem_usage1

        layer_name = model_instance.layers[layer_id].name

        # check if there is a result for this layer_id, nr_images, normalize combination
        cond_layer_id = (df_results['layer_id'] == layer_id)
        cond_model = (df_results['model'] == model)
        cond_nr_images = (df_results['nr_images'] == nr_images)
        cond_normalize = (df_results['normalize'] == normalize)
        df_previous_results = df_results[cond_layer_id & cond_model & cond_nr_images & cond_normalize]
        logger.info(f"{len(df_previous_results)} previous results for layer {layer_id}, nr_images {nr_images}, and normalize settings: {normalize}")
        if len(df_previous_results) > 0:
            logger.info(f"skip iteration - found previous results for layer {layer_id}, nr_images {nr_images}, and normalize settings: {normalize}.")
            logger.debug(f"records found: {df_previous_results}")
            continue

        # compile model, use specific layer as output
        intermediate_model = Model(inputs=model_instance.input, outputs=model_instance.get_layer(layer_name).output)
        intermediate_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=[categorical_accuracy])

        # generate image embeddings
        X_train_embeddings = intermediate_model.predict(X_train)
        X_val_embeddings = intermediate_model.predict(X_val)

        # reshape embeddings to 2D
        X_train_embeddings = X_train_embeddings.reshape(m_train, -1)
        X_val_embeddings = X_val_embeddings.reshape(m_val, -1)
        
        if normalize == "standard":
            logger.info("normalize using standard scaling")
            scaler = StandardScaler()
        elif normalize == "minmax":
            logger.info("normalize using standard scaling")
            scaler = MinMaxScaler()
        else:
            logger.info("no normalization")
            scaler = None

        if scaler:
            X_train_embeddings = scaler.fit_transform(X_train_embeddings)
            X_val_embeddings = scaler.transform(X_val_embeddings)
        
        train_score, val_score = fit_linear_model(linear_model, X_train_embeddings, y_train, X_val_embeddings, y_val)
        logger.info(f"score for layer {layer_id} - layer_name {layer_name}, train_score: {train_score:.4f}, val_score: {val_score:.4f}")
        
        # store results to csv
        row = {}
        row['run_id'] = run_id
        row['layer_id'] = layer_id
        row['layer_name'] = layer_name
        row['model'] = model
        row['nr_images'] = nr_images
        row['normalize'] = normalize
        row['train_score'] = train_score
        row['val_score'] = val_score
        df_row = pd.DataFrame(data=[row])
        df_results = pd.concat([df_results, df_row], ignore_index=True, sort=False)
        df_results.to_csv(results_csv_path, sep=";", index=False)


if __name__ == '__main__':
    if is_file:
        # provide arguments from command line
        train()
    else:
        # provide arguments in the code
        train(normalize="none", nr_images=810, model="resnet", linear_model="keras_dense")

/Users/salomon/miniconda3/envs/setdetection/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 243 images belonging to 81 classes.
Found 81 images belonging to 81 classes.
Train on 810 samples, validate on 81 samples
Epoch 1/20
810/810 [==============================] - 2s 3ms/sample - loss: 4.4670 - acc: 0.0210 - val_loss: 4.4083 - val_acc: 0.0247
Epoch 2/20
810/810 [==============================] - 0s 121us/sample - loss: 4.2495 - acc: 0.0457 - val_loss: 4.2931 - val_acc: 0.0494
Epoch 3/20
810/810 [==============================] - 0s 121us/sample - loss: 4.0581 - acc: 0.0827 - val_loss: 4.2091 - val_acc: 0.0494
Epoch 4/20
810/810 [==============================] - 0s 124us/sample - loss: 3.8826 - acc: 0.1519 - val_loss: 4.1237 - val_acc: 0.0370
Epoch 5/20
810/810 [==============================] - 0s 122us/sample - loss: 3.7164 - acc: 0.2025 - val_loss: 4.0348 - val_acc: 0.0370
Epoch 6/20
810/810 [==============================] - 0s 123us/sample - loss: 3.5599 - acc: 0.2741 - val_loss: 3.9559 - val_acc: 0.0617
Epoch 7/20
810/810 [==============================] - 0s 1

Epoch 19/20
810/810 [==============================] - 0s 303us/sample - loss: 0.0046 - acc: 1.0000 - val_loss: 1.0215 - val_acc: 0.6420
Epoch 20/20
810/810 [==============================] - 0s 280us/sample - loss: 0.0044 - acc: 1.0000 - val_loss: 1.0163 - val_acc: 0.6543
Train on 810 samples, validate on 81 samples
Epoch 1/20
810/810 [==============================] - 3s 3ms/sample - loss: 4.5754 - acc: 0.0185 - val_loss: 4.2913 - val_acc: 0.0617
Epoch 2/20
810/810 [==============================] - 0s 128us/sample - loss: 3.9593 - acc: 0.0889 - val_loss: 3.8580 - val_acc: 0.0988
Epoch 3/20
810/810 [==============================] - 0s 121us/sample - loss: 3.4584 - acc: 0.2025 - val_loss: 3.6191 - val_acc: 0.0864
Epoch 4/20
810/810 [==============================] - 0s 119us/sample - loss: 3.0249 - acc: 0.3457 - val_loss: 3.2942 - val_acc: 0.2099
Epoch 5/20
810/810 [==============================] - 0s 125us/sample - loss: 2.6263 - acc: 0.4840 - val_loss: 3.0731 - val_acc: 0.2593
Epo

810/810 [==============================] - 0s 461us/sample - loss: 0.0021 - acc: 1.0000 - val_loss: 3.2267 - val_acc: 0.4691
Epoch 19/20
810/810 [==============================] - 0s 465us/sample - loss: 0.0021 - acc: 1.0000 - val_loss: 3.2319 - val_acc: 0.4691
Epoch 20/20
810/810 [==============================] - 0s 464us/sample - loss: 0.0020 - acc: 1.0000 - val_loss: 3.2261 - val_acc: 0.4691
Train on 810 samples, validate on 81 samples
Epoch 1/20
810/810 [==============================] - 3s 3ms/sample - loss: 4.6873 - acc: 0.0346 - val_loss: 4.2135 - val_acc: 0.0494
Epoch 2/20
810/810 [==============================] - 0s 78us/sample - loss: 3.5145 - acc: 0.1531 - val_loss: 3.8650 - val_acc: 0.0864
Epoch 3/20
810/810 [==============================] - 0s 81us/sample - loss: 2.8574 - acc: 0.3037 - val_loss: 3.0365 - val_acc: 0.2593
Epoch 4/20
810/810 [==============================] - 0s 77us/sample - loss: 2.2068 - acc: 0.4951 - val_loss: 2.7458 - val_acc: 0.2593
Epoch 5/20
810/81

810/810 [==============================] - 0s 294us/sample - loss: 308.3208 - acc: 0.1543 - val_loss: 343.4574 - val_acc: 0.0247
Epoch 17/20
810/810 [==============================] - 0s 271us/sample - loss: 295.1675 - acc: 0.1765 - val_loss: 323.4164 - val_acc: 0.0741
Epoch 18/20
810/810 [==============================] - 0s 272us/sample - loss: 273.6962 - acc: 0.2210 - val_loss: 300.7693 - val_acc: 0.0494
Epoch 19/20
810/810 [==============================] - 0s 271us/sample - loss: 232.2205 - acc: 0.1827 - val_loss: 239.5717 - val_acc: 0.0494
Epoch 20/20
810/810 [==============================] - 0s 276us/sample - loss: 188.5798 - acc: 0.1901 - val_loss: 199.0577 - val_acc: 0.0864
Train on 810 samples, validate on 81 samples
Epoch 1/20
810/810 [==============================] - 4s 4ms/sample - loss: 753950.5491 - acc: 0.0123 - val_loss: 2670697.7500 - val_acc: 0.0247
Epoch 2/20
810/810 [==============================] - 1s 1ms/sample - loss: 2563109.1994 - acc: 0.0444 - val_loss: 37

...

810/810 [==============================] - 0s 446us/sample - loss: 1.3868 - acc: 0.7333 - val_loss: 3.3248 - val_acc: 0.1605
Epoch 12/20
810/810 [==============================] - 0s 471us/sample - loss: 1.1600 - acc: 0.8062 - val_loss: 3.0647 - val_acc: 0.1605
Epoch 13/20
810/810 [==============================] - 0s 467us/sample - loss: 0.8898 - acc: 0.8728 - val_loss: 2.9317 - val_acc: 0.2346
Epoch 14/20
810/810 [==============================] - 0s 476us/sample - loss: 0.7603 - acc: 0.9185 - val_loss: 3.4265 - val_acc: 0.1728
Epoch 15/20
810/810 [==============================] - 0s 449us/sample - loss: 0.7194 - acc: 0.9049 - val_loss: 3.0195 - val_acc: 0.1975
Epoch 16/20
810/810 [==============================] - 0s 454us/sample - loss: 0.6113 - acc: 0.9531 - val_loss: 2.8189 - val_acc: 0.1852
Epoch 17/20
810/810 [==============================] - 0s 442us/sample - loss: 0.5507 - acc: 0.9580 - val_loss: 2.9093 - val_acc: 0.2222
Epoch 18/20
810/810 [==============================] 

810/810 [==============================] - 0s 256us/sample - loss: 1126.0902 - acc: 0.0543 - val_loss: 1171.3263 - val_acc: 0.0370
Epoch 10/20
810/810 [==============================] - 0s 285us/sample - loss: 985.9878 - acc: 0.0741 - val_loss: 981.3900 - val_acc: 0.0123
Epoch 11/20
810/810 [==============================] - 0s 274us/sample - loss: 801.3464 - acc: 0.0852 - val_loss: 735.4407 - val_acc: 0.0247
Epoch 12/20
810/810 [==============================] - 0s 243us/sample - loss: 593.3449 - acc: 0.1185 - val_loss: 579.5615 - val_acc: 0.0741
Epoch 13/20
810/810 [==============================] - 0s 252us/sample - loss: 438.4677 - acc: 0.1469 - val_loss: 507.7661 - val_acc: 0.0247
Epoch 14/20
810/810 [==============================] - 0s 252us/sample - loss: 369.2918 - acc: 0.2000 - val_loss: 380.3401 - val_acc: 0.0617
Epoch 15/20
810/810 [==============================] - 0s 256us/sample - loss: 300.2032 - acc: 0.2346 - val_loss: 348.2784 - val_acc: 0.0617
Epoch 16/20
810/810 [==

810/810 [==============================] - 0s 112us/sample - loss: 2.9858 - acc: 0.4765 - val_loss: 3.2924 - val_acc: 0.1975
Epoch 7/20
810/810 [==============================] - 0s 102us/sample - loss: 2.7743 - acc: 0.5691 - val_loss: 3.1287 - val_acc: 0.2716
Epoch 8/20
810/810 [==============================] - 0s 104us/sample - loss: 2.5700 - acc: 0.6309 - val_loss: 3.0179 - val_acc: 0.3210
Epoch 9/20
810/810 [==============================] - 0s 107us/sample - loss: 2.3920 - acc: 0.6815 - val_loss: 2.8849 - val_acc: 0.3580
Epoch 10/20
810/810 [==============================] - 0s 107us/sample - loss: 2.2235 - acc: 0.7370 - val_loss: 2.7719 - val_acc: 0.4074
Epoch 11/20
810/810 [==============================] - 0s 102us/sample - loss: 2.0769 - acc: 0.7889 - val_loss: 2.6708 - val_acc: 0.4074
Epoch 12/20
810/810 [==============================] - 0s 103us/sample - loss: 1.9413 - acc: 0.8247 - val_loss: 2.5733 - val_acc: 0.4568
Epoch 13/20
810/810 [==============================] - 0

Epoch 5/20
810/810 [==============================] - 0s 99us/sample - loss: 3.4242 - acc: 0.1901 - val_loss: 3.6778 - val_acc: 0.1358
Epoch 6/20
810/810 [==============================] - 0s 95us/sample - loss: 3.2106 - acc: 0.2667 - val_loss: 3.5190 - val_acc: 0.1235
Epoch 7/20
810/810 [==============================] - 0s 95us/sample - loss: 3.0068 - acc: 0.3185 - val_loss: 3.4504 - val_acc: 0.1358
Epoch 8/20
810/810 [==============================] - 0s 90us/sample - loss: 2.8355 - acc: 0.3951 - val_loss: 3.3058 - val_acc: 0.1728
Epoch 9/20
810/810 [==============================] - 0s 114us/sample - loss: 2.6889 - acc: 0.4383 - val_loss: 3.2514 - val_acc: 0.1111
Epoch 10/20
810/810 [==============================] - 0s 99us/sample - loss: 2.5290 - acc: 0.5025 - val_loss: 3.1360 - val_acc: 0.1975
Epoch 11/20
810/810 [==============================] - 0s 97us/sample - loss: 2.4117 - acc: 0.5111 - val_loss: 3.0692 - val_acc: 0.1728
Epoch 12/20
810/810 [==============================]

810/810 [==============================] - 0s 149us/sample - loss: 37.8509 - acc: 0.0840 - val_loss: 45.5080 - val_acc: 0.0370
Epoch 4/20
810/810 [==============================] - 0s 174us/sample - loss: 38.0858 - acc: 0.1235 - val_loss: 44.5592 - val_acc: 0.0494
Epoch 5/20
810/810 [==============================] - 0s 170us/sample - loss: 37.0008 - acc: 0.1593 - val_loss: 42.1421 - val_acc: 0.0864
Epoch 6/20
810/810 [==============================] - 0s 172us/sample - loss: 32.9069 - acc: 0.1728 - val_loss: 36.2641 - val_acc: 0.0494
Epoch 7/20
810/810 [==============================] - 0s 165us/sample - loss: 24.0338 - acc: 0.2333 - val_loss: 29.1589 - val_acc: 0.0247
Epoch 8/20
810/810 [==============================] - 0s 169us/sample - loss: 19.5243 - acc: 0.3309 - val_loss: 22.9684 - val_acc: 0.0741
Epoch 9/20
810/810 [==============================] - 0s 171us/sample - loss: 12.8124 - acc: 0.4185 - val_loss: 17.8415 - val_acc: 0.0741
Epoch 10/20
810/810 [========================